Lo primero que hacemos es dividir el dataset original en dos partes. 
<BR>

i) Incluye los nulos (10.415) que será donde hagamos las prediciones.
<BR>

ii) No incluye los nulos (21.229) y será el que utilicemos para entrenar el modelo.  

In [1]:
import csv
import pandas as pd
import matplotlib.pyplot as plt

data = pd.read_csv('data/data.csv')

data_copia = data.copy()

# Crear un DataFrame con filas que contienen NaN en 'x_e_out [-]'

df_nan = data_copia[data_copia['x_e_out [-]'].isnull()]

# Crear un DataFrame sin filas que contienen NaN en 'x_e_out [-]'

df_sin_nan = data_copia[~data_copia['x_e_out [-]'].isnull()]

print("Data_copia: ", data_copia.shape)
print("Data con los NUN:", df_nan.shape)
print("Data sin los NUN: ", df_sin_nan.shape)

Data_copia:  (31644, 10)
Data con los NUN: (10415, 10)
Data sin los NUN:  (21229, 10)


In [2]:
# Calcular la media de la columna "x_e_out [-]"
media_x_e_out = df_sin_nan['x_e_out [-]'].mean()

print("Media de x_e_out [-]:", media_x_e_out)

Media de x_e_out [-]: -0.0004528625936219324


In [8]:
# Realizar la codificación one-hot en la columna "author" y "geometry". 

data_encoded = pd.get_dummies(df_sin_nan, columns=['author', "geometry"])

data_encoded.shape


(21229, 21)

# Probamos el modelo de XG_Boost

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Reemplazar los valores faltantes con la media de la columna correspondiente
data_encoded.fillna(data_encoded.mean(), inplace=True)

# Separar las características (X) y la variable objetivo (y)
X = data_encoded.drop('x_e_out [-]', axis=1)
y = data_encoded['x_e_out [-]']

# Dividir el dataset en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear un objeto Scaler y ajustarlo a los datos de entrenamiento
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Crear y entrenar el modelo XGBoost
modelo_XGB = XGBRegressor()
modelo_XGB.fit(X_train_scaled, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = modelo_XGB.predict(X_test_scaled)

# Calcular el RMSE
rmse = mean_squared_error(y_test, y_pred, squared=False)

# Calcular el R cuadrado
r2 = r2_score(y_test, y_pred)

# Calcular el MSE
mse = mean_squared_error(y_test, y_pred)

# Calcular el MAE
mae = mean_absolute_error(y_test, y_pred)

print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared:", r2)
print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)

Root Mean Squared Error (RMSE): 0.07724510934181494
R-squared: 0.4125767407412997
Mean Squared Error (MSE): 0.005966806917228946
Mean Absolute Error (MAE): 0.05393318578209327


# Ponemos en el mismo formato el dataset NAN y hacemos la predicción. 

In [7]:
# Realizar la codificación one-hot en la columna "author" y "geometry"

data_encoded_NUN = pd.get_dummies(df_nan, columns=['author', "geometry"])

# Reemplazar los valores faltantes con la media de la columna correspondiente
data_encoded_NUN.fillna(data_encoded_NUN.mean(), inplace=True)

# Separar las características (X_new) del nuevo DataFrame
X_new = data_encoded_NUN.drop('x_e_out [-]', axis=1)

# Escalar los datos del nuevo DataFrame utilizando el mismo objeto Scaler ajustado previamente
X_new_scaled = scaler.transform(X_new)

# Realizar la predicción en el nuevo DataFrame
y_new_pred = modelo_XGB.predict(X_new_scaled)

# Agregar los valores predichos al nuevo DataFrame
data_encoded_NUN['x_e_out [-]'] = y_new_pred

# Imprimir el nuevo DataFrame con los valores predichos

formato_subm = data_encoded_NUN[['id', 'x_e_out [-]']]

# Convertir el DataFrame seleccionado a CSV
formato_subm.to_csv('Submission_7.csv', index=False)

In [ ]:

# Ajustamos los hiperparametros con GridSearchCV

import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

# Reemplazar los valores faltantes con la media de la columna correspondiente
data_encoded.fillna(data_encoded.mean(), inplace=True)

# Separar las características (X) y la variable objetivo (y)
X = data_encoded.drop('x_e_out [-]', axis=1)
y = data_encoded['x_e_out [-]']

# Dividir el dataset en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear un objeto Scaler y ajustarlo a los datos de entrenamiento
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Definir los hiperparámetros a ajustar
parameters = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.3],
    'max_depth': [3, 4, 5]
}

# Crear el modelo base
base_model = XGBRegressor()

# Crear el objeto GridSearchCV
grid_search = GridSearchCV(estimator=base_model, param_grid=parameters, scoring='neg_root_mean_squared_error', cv=5)

# Realizar la búsqueda de hiperparámetros
grid_search.fit(X_train_scaled, y_train)

# Obtener el mejor modelo y los mejores parámetros
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

# Realizar predicciones en el conjunto de prueba con el mejor modelo
y_pred = best_model.predict(X_test_scaled)

# Calcular el RMSE con el mejor modelo
rmse = mean_squared_error(y_test, y_pred, squared=False)

print("Best Parameters:", best_params)
print("Root Mean Squared Error (RMSE):", rmse)
